##### Bibliotecas utilizadas

In [18]:
import requests 
import pandas as pd
import numpy as np
import time
import random
import csv
import copy

##### Gerando dados

In [ ]:
# Utilizado caso ainda não exista uma base de dados

df_final = pd.DataFrame()
contador = 1 
for i in range (4):
    dados = requests.get('http://localhost:3000/api/ep1').json()
    df = pd.DataFrame(dados)
    df['semana']  = contador
    df_final = pd.concat([df, df_final], axis = 0)
    contador +=1



##### Tratando dados

In [ ]:
# Ajustando data 
df_final['date'] = df_final['date'].map(lambda x: time.ctime(x))

In [9]:
# Ordenando dados 
df_final.sort_values('semana', axis = 0, ascending = True, inplace = True)

In [ ]:
# Ajustando valores negativos e NaN e convertendo os números em inteiro
colunas = df_final.columns

for coluna in colunas[2:len(colunas)-1]:
        df_final[coluna] = df_final[coluna].fillna(0) 
        df_final[coluna] = df_final[coluna].apply(lambda x: int(abs(x)))
        

In [85]:
# Ajustando nomes das colunas para a comparação entre elas funcionar. 
df_final.rename(columns={'prod_0':'prod_00', 'prod_1':'prod_01', 'prod_2':'prod_02', \
    'prod_3':'prod_03', 'prod_4':'prod_04', 'prod_5':'prod_05',\
        'prod_6':'prod_06', 'prod_7':'prod_07', 'prod_8':'prod_08', \
        'prod_9':'prod_09'}, inplace = True)

In [65]:
# gerando csv
df_final.to_csv('all_sales.csv', encoding = 'iso-8859-1', sep = ';')

##### Tratando preços

In [84]:
# Quando já existir um csv na base, abrir arquivo.

df_final = pd.read_csv('all_sales.csv', encoding = 'iso-8859-1', sep = ';')

In [85]:
# Gerando preços aleatórios para produtos novos e segunda semana. 

df_precos = copy.deepcopy(df_final) 

# Apagando todas as linhas do dataframe copiado.
df_precos.drop(list(range(len(df_final))), axis = 0, inplace = True, errors= 'ignore')

# Apagando colunas que não serão necessárias. 
df_precos.drop(['id', 'date', 'total_por_unidade', 'total_por_kg', \
    'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], axis = 1, errors= 'ignore', inplace = True)

df_precos.loc[0] = [random.randint(10,30) for x in range(df_precos.shape[1])]
df_precos.loc[1] = df_precos.loc[0]
df_precos['semana'].loc[0] = 1
df_precos['semana'].loc[1] = 2



In [105]:
df_precos

,prod_01,prod_02,prod_03,prod_05,prod_08,prod_09,prod_10,prod_15,prod_00,prod_04,prod_06,prod_11,prod_07,semana
0,16,15,11,22,13,16,18,20,20,15,19,23,18,1
1,16,15,11,22,13,16,18,20,20,15,19,23,18,2


In [106]:
# Verificando o número de semanas

df_consulta = pd.read_csv('all_sales.csv', sep =';', encoding = 'iso-8859-1')

df_consulta = df_consulta.groupby('semana').mean().diff()

f_v = 0.5 + 1/(1+np.exp(1)**-df_consulta)

f_v.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace = True)

f_v.drop([1],axis=0,inplace = True)

f_v.reset_index(inplace= True)




C:\Users\gusta\AppData\Local\Temp/ipykernel_16792/3808266037.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_consulta = df_consulta.groupby('semana').mean().diff()


In [112]:
teste = 0.5 + 1/(1+np.exp(1)**-0.5)
teste * 10

11.224593312018545

In [108]:
df_precos

,prod_01,prod_02,prod_03,prod_05,prod_08,prod_09,prod_10,prod_15,prod_00,prod_04,prod_06,prod_11,prod_07,semana
0,16,15,11,22,13,16,18,20,20,15,19,23,18,1
1,16,15,11,22,13,16,18,20,20,15,19,23,18,2


In [126]:
# Gerando valores para as demais semanas

f_v


,semana,prod_01,prod_02,prod_03,prod_05,prod_08,prod_09,prod_10,prod_15,prod_00,prod_04,prod_06,prod_11,prod_07
0,2,0.786854,1.309998,0.500022,0.542814,1.500000,1.5,1.5,0.5,0.500112,1.490118,0.501051,1.5,1.499822
1,3,0.511640,1.498248,0.524967,1.110176,1.500000,1.5,1.5,0.5,1.499791,1.145062,0.605001,1.5,0.567813
2,4,1.425047,0.502363,1.337040,1.499475,0.690302,0.5,0.5,0.5,0.500016,1.486071,1.499729,0.5,0.830087


In [ ]:
x = 2
v = 0

for coluna in df_precos.iteritems():
    df_precos.loc[2] = f_v.loc[0] * df_precos.loc[1]

##### Gerando transações do mês

In [198]:
df_vendas = df_final.copy()

In [199]:
# Reduzindo data
df_vendas['date'] = df_vendas['date'].apply(lambda x: str(x)[4:7] + '-' + str(x)[20:24])


In [200]:
# Eliminando colunas
df_vendas.drop(['Unnamed: 0', 'id','semana'], axis = 1, inplace = True, errors='ignore')


In [201]:
df_vendas = df_vendas.groupby('date').sum()

In [202]:
cabecalho = df_vendas.columns

cabecalho_unidades = list(cabecalho[cabecalho <= 'prod_07'])
cabecalho_kg = list(cabecalho[cabecalho >= 'prod_07'])


In [96]:
# Gerando csv 
df_vendas.to_csv('monthly_sales.csv', encoding = 'iso-8859-1', sep = ';')

##### Salvando arquivo CSV

In [237]:
df_precos.to_csv('weekly_price.csv', encoding = 'iso-8859-1', sep = ';')
    